In [6]:
import cv2
import numpy as np
import dlib

cap = cv2.VideoCapture(0)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame , cv2.COLOR_BGR2GRAY)
    
    faces = detector(gray)
    for face in faces:
        #print (face)
        #x , y = face.left() , face.top()
        #x1 , y1 = face.right() , face.bottom()
        #cv2.rectangle(frame , (x , y) , (x1 , y1) , (0 , 255 , 0) , 2)
        landmarks = predictor(gray , face)
        #x = landmarks.part(36).x
        #y = landmarks.part(36).y
        #cv2.circle(frame , (x , y) , 3 , (0 , 0 , 255) , 2)
        left_point = (landmarks.part(36).x , landmarks.part(36).y)
        right_point = (landmarks.part(39).x , landmarks.part(39).y)
        
        mid_point_x = int((landmarks.part(36).x + landmarks.part(39).x)/2)
        mid_point_y = int((landmarks.part(36).y + landmarks.part(39).y)/2)
        
        hor_line = cv2.line(frame , left_point , right_point , (0 , 255 , 0) , 1)
        
        top_point = (int((landmarks.part(37).x + landmarks.part(38).x)/2)  , int((landmarks.part(37).y + landmarks.part(38).y)/2))
        bottom_point = (int((landmarks.part(41).x + landmarks.part(40).x)/2)  , int((landmarks.part(41).y + landmarks.part(40).y)/2))
        
        ver_line = cv2.line(frame , top_point , bottom_point , (0 , 255 , 0) , 1)
        
        #x = int((landmarks.part(36).x + landmarks.part(39).x)/2)
        #y = int((((landmarks.part(37).y + landmarks.part(38).y)/2) + ((landmarks.part(37).y + landmarks.part(38).y)/2))/2)
        
        cv2.circle(frame , (mid_point_x , mid_point_y) , 1 , (255 , 0 , 0) , 1)
    
    cv2.imshow("Frame" , frame)
    
    key = cv2.waitKey(1)
    if key == 27:
        break
        
cap.release()
cv2.destroyAllWindows()